# Implementing Recommender Systems - Lab

## Introduction

In this lab, you'll practice creating a recommender system model using `surprise`. You'll also get the chance to create a more complete recommender system pipeline to obtain the top recommendations for a specific user.


## Objectives

In this lab you will: 

- Use surprise's built-in reader class to process data to work with recommender algorithms 
- Obtain a prediction for a specific user for a particular item 
- Introduce a new user with rating to a rating matrix and make recommendations for them 
- Create a function that will return the top n recommendations for a user 


For this lab, we will be using the famous 1M movie dataset. It contains a collection of user ratings for many different movies. In the last lesson, you were exposed to working with `surprise` datasets. In this lab, you will also go through the process of reading in a dataset into the `surprise` dataset format. To begin with, load the dataset into a Pandas DataFrame. Determine which columns are necessary for your recommendation system and drop any extraneous ones.

In [1]:
import pandas as pd
df = pd.read_csv('./ml-latest-small/ratings.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
userId       100836 non-null int64
movieId      100836 non-null int64
rating       100836 non-null float64
timestamp    100836 non-null int64
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [3]:
# Drop unnecessary columns
new_df = df.drop('timestamp', axis=1)
new_df.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


It's now time to transform the dataset into something compatible with `surprise`. In order to do this, you're going to need `Reader` and `Dataset` classes. There's a method in `Dataset` specifically for loading dataframes.

In [9]:
from surprise import Reader, Dataset
reader = Reader()
data = Dataset.load_from_df(new_df,reader)
type(data)



surprise.dataset.DatasetAutoFolds

Let's look at how many users and items we have in our dataset. If using neighborhood-based methods, this will help us determine whether or not we should perform user-user or item-item similarity

In [10]:
dataset = data.build_full_trainset()
print('Number of users: ', dataset.n_users, '\n')
print('Number of items: ', dataset.n_items)

Number of users:  610 

Number of items:  9724


## Determine the best model 

Now, compare the different models and see which ones perform best. For consistency sake, use RMSE to evaluate models. Remember to cross-validate! Can you get a model with a higher average RMSE on test data than 0.869?

In [11]:
# importing relevant libraries
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms import SVD
from surprise.prediction_algorithms import KNNWithMeans, KNNBasic, KNNBaseline
from surprise.model_selection import GridSearchCV
import numpy as np

In [12]:
## Perform a gridsearch with SVD
# ⏰ This cell may take several minutes to run
params = {'n_factors': [20, 50, 100],
         'reg_all': [0.02, 0.05, 0.1]}
g_s_svd = GridSearchCV(SVD,param_grid=params,n_jobs=-1)
g_s_svd.fit(data)
print(g_s_svd.best_score)
print(g_s_svd.best_params)


{'rmse': 0.868239582287875, 'mae': 0.6678423459903207}
{'rmse': {'n_factors': 100, 'reg_all': 0.05}, 'mae': {'n_factors': 50, 'reg_all': 0.05}}


In [12]:
# print out optimal parameters for SVD after GridSearch

{'rmse': 0.8689250510051669, 'mae': 0.6679404366294037}
{'rmse': {'n_factors': 50, 'reg_all': 0.05}, 'mae': {'n_factors': 100, 'reg_all': 0.05}}


In [13]:
# cross validating with KNNBasic
knn_basic = KNNBasic(sim_options={'name':'pearson', 'user_based':True})
cv_knn_basic = cross_validate(knn_basic, data, n_jobs=-1)


for i in cv_knn_basic.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_basic['test_rmse']))


('test_rmse', array([0.97465456, 0.96959474, 0.97525393, 0.96669608, 0.97310709]))
('test_mae', array([0.75429414, 0.7482595 , 0.74819378, 0.74565157, 0.75484205]))
('fit_time', (1.173002004623413, 1.3780016899108887, 1.2739989757537842, 1.2819976806640625, 1.143000841140747))
('test_time', (2.5590012073516846, 2.5160014629364014, 2.343008041381836, 2.0059993267059326, 1.833019495010376))
-----------------------
0.9718612808162277


In [15]:
# print out the average RMSE score for the test set

('test_rmse', array([0.97646619, 0.97270627, 0.97874535, 0.97029184, 0.96776748]))
('test_mae', array([0.75444119, 0.75251222, 0.7531242 , 0.74938542, 0.75152129]))
('fit_time', (0.46678805351257324, 0.54010009765625, 0.7059998512268066, 0.5852491855621338, 1.0139541625976562))
('test_time', (2.308177947998047, 2.4834508895874023, 2.6563329696655273, 2.652374029159546, 1.2219891548156738))
-----------------------
0.9731954260849399


In [14]:
# cross validating with KNNBaseline
knn_baseline = KNNBaseline(sim_options={'name':'pearson', 'user_based':True})
cv_knn_baseline = cross_validate(knn_baseline,data)

Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.


In [15]:
# print out the average score for the test set
for i in cv_knn_baseline.items():
    print(i)

np.mean(cv_knn_baseline['test_rmse'])

('test_rmse', array([0.88247145, 0.8798947 , 0.86132047, 0.88253577, 0.88388647]))
('test_mae', array([0.67379086, 0.67208689, 0.65833621, 0.6732847 , 0.67669567]))
('fit_time', (1.2709953784942627, 1.2530150413513184, 1.2880005836486816, 1.2639734745025635, 1.2319951057434082))
('test_time', (2.5870211124420166, 2.6609930992126465, 2.5309979915618896, 2.5660197734832764, 2.523995876312256))


0.8780217731786062

Based off these outputs, it seems like the best performing model is the SVD model with `n_factors = 50` and a regularization rate of 0.05. Use that model or if you found one that performs better, feel free to use that to make some predictions.

## Making Recommendations

It's important that the output for the recommendation is interpretable to people. Rather than returning the `movie_id` values, it would be far more valuable to return the actual title of the movie. As a first step, let's read in the movies to a dataframe and take a peek at what information we have about them.

In [16]:
df_movies = pd.read_csv('./ml-latest-small/movies.csv')

In [17]:
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


## Making simple predictions
Just as a reminder, let's look at how you make a prediction for an individual user and item. First, we'll fit the SVD model we had from before.

In [18]:
svd = SVD(n_factors= 50, reg_all=0.05)
svd.fit(dataset)

In [19]:
svd.predict(2, 4)

Prediction(uid=2, iid=4, r_ui=None, est=2.954188618148457, details={'was_impossible': False})

This prediction value is a tuple and each of the values within it can be accessed by way of indexing. Now let's put our knowledge of recommendation systems to do something interesting: making predictions for a new user!

## Obtaining User Ratings 

It's great that we have working models and everything, but wouldn't it be nice to get to recommendations specifically tailored to your preferences? That's what we'll be doing now. The first step is to create a function that allows us to pick randomly selected movies. The function should present users with a movie and ask them to rate it. If they have not seen the movie, they should be able to skip rating it. 

The function `movie_rater()` should take as parameters: 

* `movie_df`: DataFrame - a dataframe containing the movie ids, name of movie, and genres
* `num`: int - number of ratings
* `genre`: string - a specific genre from which to draw movies

The function returns:
* rating_list : list - a collection of dictionaries in the format of {'userId': int , 'movieId': int , 'rating': float}

#### This function is optional, but fun :) 

In [20]:
def movie_rater(movie_df,num, genre=None):
    userID = 1000
    rating_list = []
    while num > 0:
        if genre:
            movie = movie_df[movie_df['genres'].str.contains(genre)].sample(1)
        else:
            movie = movie_df.sample(1)
        print(movie)
        rating = input('How do you rate this movie on a scale of 1-5, press n if you have not seen :\n')
        if rating == 'n':
            continue
        else:
            rating_one_movie = {'userId':userID,'movieId':movie['movieId'].values[0],'rating':rating}
            rating_list.append(rating_one_movie) 
            num -= 1
    return rating_list      

        

In [21]:
# try out the new function here!
user_rating = movie_rater(df_movies, 4, 'Comedy')

      movieId                     title        genres
8235   104211  We're the Millers (2013)  Comedy|Crime
How do you rate this movie on a scale of 1-5, press n if you have not seen :
n
      movieId          title  genres
7585    86293  Arthur (2011)  Comedy
How do you rate this movie on a scale of 1-5, press n if you have not seen :
4
      movieId                                   title  \
3511     4799  It's a Mad, Mad, Mad, Mad World (1963)   

                             genres  
3511  Action|Adventure|Comedy|Crime  
How do you rate this movie on a scale of 1-5, press n if you have not seen :
4
      movieId                 title                genres
3291     4451  Jump Tomorrow (2001)  Comedy|Drama|Romance
How do you rate this movie on a scale of 1-5, press n if you have not seen :
n
      movieId                    title  genres
6908    64032  Four Christmases (2008)  Comedy
How do you rate this movie on a scale of 1-5, press n if you have not seen :
n
      movieId         

##### If you're struggling to come up with the above function, you can use this list of user ratings to complete the next segment

In [22]:
user_rating

[{'userId': 1000, 'movieId': 86293, 'rating': '4'},
 {'userId': 1000, 'movieId': 4799, 'rating': '4'},
 {'userId': 1000, 'movieId': 26152, 'rating': '5'},
 {'userId': 1000, 'movieId': 1821, 'rating': '2'}]

### Making Predictions With the New Ratings
Now that you have new ratings, you can use them to make predictions for this new user. The proper way this should work is:

* add the new ratings to the original ratings DataFrame, read into a `surprise` dataset 
* train a model using the new combined DataFrame
* make predictions for the user
* order those predictions from highest rated to lowest rated
* return the top n recommendations with the text of the actual movie (rather than just the index number) 

In [23]:
## add the new ratings to the original ratings DataFrame
new_ratings_df = new_df.append(user_rating,ignore_index=True)
new_data = Dataset.load_from_df(new_ratings_df,reader)


In [24]:
# train a model using the new combined DataFrame
svd_ = SVD(n_factors= 50, reg_all=0.05)
svd_.fit(new_data.build_full_trainset())


In [25]:
# make predictions for the user
# you'll probably want to create a list of tuples in the format (movie_id, predicted_score)
list_of_movies = []
for m_id in new_df['movieId'].unique():
    list_of_movies.append( (m_id,svd_.predict(1000,m_id)[3]))
list_of_movies[:10]

[(1, 4.000579445773636),
 (3, 3.3121020385552775),
 (6, 4.040394264495878),
 (47, 4.119654573845358),
 (50, 4.350010265897187),
 (70, 3.760533935073498),
 (101, 4.087012492892429),
 (110, 4.07383253021915),
 (151, 3.6930968188976316),
 (157, 3.326001111354192)]

In [26]:
# order the predictions from highest to lowest rated

# order the predictions from highest to lowest rated
ranked_movies = sorted(list_of_movies, key=lambda x:x[1], reverse=True)
ranked_movies[:10]

[(750, 4.5745024566502375),
 (1204, 4.56780958737071),
 (56782, 4.5350750821477455),
 (318, 4.4864598441297785),
 (48516, 4.478554660868837),
 (2959, 4.451247176404058),
 (7361, 4.447396785894376),
 (1203, 4.445679024257798),
 (296, 4.441826697550401),
 (1213, 4.438716722652014)]

 For the final component of this challenge, it could be useful to create a function `recommended_movies()` that takes in the parameters:
* `user_ratings`: list - list of tuples formulated as (user_id, movie_id) (should be in order of best to worst for this individual)
* `movie_title_df`: DataFrame 
* `n`: int - number of recommended movies 

The function should use a `for` loop to print out each recommended *n* movies in order from best to worst

In [27]:
# return the top n recommendations using the 
def recommended_movies(user_ratings,movie_title_df,n):
        for idx, rec in enumerate(user_ratings):
            title = movie_title_df.loc[movie_title_df['movieId'] == int(rec[0])]['title']
            print('Recommendation # ', idx+1, ': ', title, '\n')
            n-= 1
            if n == 0:
                break
            
recommended_movies(ranked_movies,df_movies,5)

Recommendation #  1 :  602    Dr. Strangelove or: How I Learned to Stop Worr...
Name: title, dtype: object 

Recommendation #  2 :  906    Lawrence of Arabia (1962)
Name: title, dtype: object 

Recommendation #  3 :  6648    There Will Be Blood (2007)
Name: title, dtype: object 

Recommendation #  4 :  277    Shawshank Redemption, The (1994)
Name: title, dtype: object 

Recommendation #  5 :  6315    Departed, The (2006)
Name: title, dtype: object 



## Level Up (Optional)

* Try and chain all of the steps together into one function that asks users for ratings for a certain number of movies, then all of the above steps are performed to return the top $n$ recommendations
* Make a recommender system that only returns items that come from a specified genre

## Summary

In this lab, you got the chance to implement a collaborative filtering model as well as retrieve recommendations from that model. You also got the opportunity to add your own recommendations to the system to get new recommendations for yourself! Next, you will learn how to use Spark to make recommender systems.